<a href="https://colab.research.google.com/github/adarshsaji/finetuned-gemma2-kagglex24-fellowship/blob/main/Chatbot_devlopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface
!pip install keras_nlp
!pip install kagglehub
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [2]:
import os
import kagglehub
import keras_nlp
import keras_hub

In [3]:
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [5]:
causal_lm = keras_nlp.models.CausalLM.from_preset("kaggle://adarshsaji/gemma/keras/gemma-startup-engine")

100%|██████████| 782/782 [00:00<00:00, 946kB/s]


100%|██████████| 2.91k/2.91k [00:00<00:00, 3.27MB/s]


100%|██████████| 4.04M/4.04M [00:01<00:00, 2.24MB/s]


100%|██████████| 9.74G/9.74G [10:17<00:00, 16.9MB/s]


In [10]:
template = "Instruction:\n{Instruction}\n\nResponse:\n{Response}"

In [15]:
prompt = template.format(
    Instruction="What is the market size and potential for healthcare billing systems, and how can startups capitalize on this opportunity?",
    Response="",
)

response = causal_lm.generate(prompt, max_length=2048)

In [16]:
print(response)

Instruction:
What is the market size and potential for healthcare billing systems, and how can startups capitalize on this opportunity?

Response:
The market size for healthcare billing systems is estimated to be $1.5 billion, with a compound annual growth rate of 10%. Startups can capitalize on this opportunity by developing innovative solutions that address the inefficiencies and challenges in healthcare billing, such as the use of AI and machine learning to automate processes and improve accuracy.




---



In [124]:
import keras_nlp
import tensorflow as tf
from typing import List, Tuple
import gradio as gr
import time

class ChatInterface:
    def __init__(self):
        self.model = causal_lm
        self.chat_history: List[Tuple[str, str]] = []
        self.system_prompt = """
You are an AI Tech Consultant, skilled in simplifying technical processes for non-technical users. Guide them in turning ideas into products by offering practical, clear advice on:

- Feasibility: Help assess idea feasibility, complexity, and time commitment.
- Product Development: Outline the step-by-step journey from idea to launch, including prototyping, testing, and iteration.
- Tech Basics: Simplify front-end, back-end, full-stack, and data technologies, explaining their roles in simple terms.
- Cost Estimation: Set realistic expectations for costs, covering team size, tech stack, time, and hidden factors.
- Tech Recommendations: Suggest suitable technologies and best practices for their idea (e.g., frameworks, cloud solutions).
- Collaboration: Advise on team roles (e.g., front-end developers, back-end engineers) and effective communication strategies.
- Clarity: Avoid jargon; use analogies and simple language to build user confidence.
- If a question is asked out of the above said context respond I can't answer this question.

Think step by step, use careful reasoning.

"""
        self.MAX_HISTORY = 5  # Maximum number of previous exchanges to keep
        self.MIN_CHARS = 10  # Minimum characters required for submission

    async def process_message(self, message: str) -> str:
        # Format messages for the model
        formatted_history = self.format_history()
        prompt = f"{self.system_prompt}\n\n{formatted_history}Instruction:\n{message}\n\nResponse:\n"

        # Generate response
        response = self.generate_response(prompt)

        # Update chat history
        self.update_history(message, response)

        return response

    def format_history(self) -> str:
        return "".join(f"Instruction:\n{human}\n\nResponse:\n{assistant}\n\n"
                       for human, assistant in self.chat_history[-self.MAX_HISTORY:])

    def generate_response(self, prompt: str) -> str:
        template = "Instruction:\n{Instruction}\n\nResponse:\n{Response}"
        input_text = template.format(Instruction=prompt, Response="")

        outputs = self.model.generate(input_text, max_length=1000, strip_prompt=True)

        return outputs

    def update_history(self, message: str, response: str):
        self.chat_history.append((message, response))
        if len(self.chat_history) > self.MAX_HISTORY:
            self.chat_history = self.chat_history[-self.MAX_HISTORY:]

    def create_interface(self):
        with gr.Blocks(css="footer {visibility: hidden}") as interface:
            gr.Markdown("<h1 style='text-align: center;'>🤖 Startup Buddy</h1>")
            gr.Markdown("<p style='text-align: center;'>I can answer your startup/tech world related questions!</p>")
            chatbot = gr.Chatbot(
                height=500,
                show_label=False,
                container=True,
                scale=1,
                value=[[None, "Here's what I can do to help you on your product development journey:\n\n" +
                       "* Answer recent tech/start-up related questions precisely between October 2023 - October 2024. \n" +
                       "* Try `What is the current status of Waymo's commercial robotaxi operations in California?` or `What is the market size and potential for healthcare billing systems, and how can startups capitalize on this opportunity?` * \n" +
                       "* Understand your market: I can analyze trends, funding patterns, and competitor activity to help you figure out if your product idea has a good chance of success.\n" +
                       "* Map out your path: I'll guide you through the entire process, from brainstorming to launch, with clear steps and actionable advice.\n" +
                       "* Know your competition: I can identify your potential rivals, highlight their strengths and weaknesses, and suggest ways to make your product stand out.\n" +
                       "* Attract investors: I can share insights on recent funding rounds, investor preferences, and valuation trends to help you prepare for fundraising.\n" +
                       "* Choose the right tech: I'll recommend technologies and practices that are popular and effective in the startup world.\n" +
                       "* Anticipate challenges: I can point out potential roadblocks based on common startup struggles and suggest strategies to overcome them.\n" +
                       "* Plan for growth: I can share successful growth tactics used by other startups to help you scale your business.\n\n" +
                       "Essentially, I'm here to make the complex world of product development more understandable and manageable for you.\n\n" +
                       "What's your product idea? Let's explore it together!"]],
                show_copy_button=True,
            )

            with gr.Row():
                msg = gr.Textbox(
                    label="Type your message here...",
                    placeholder="Type your message here...",
                    show_label=False,
                    container=True,
                )
            with gr.Row():
                submit_btn = gr.Button(
                    "Submit",
                    size="lg",
                )
                clear = gr.Button(
                    "Clear Chat",
                    size="lg"
                )

            async def respond(message, chat_history):
                if len(message) < self.MIN_CHARS:
                    return message, chat_history

                # Clear the initial message when the first user message is sent
                if len(chat_history) == 1 and chat_history[0][0] is None:
                    chat_history = []
                bot_message = await self.process_message(message)
                chat_history.append((message, bot_message))
                return "", chat_history

            def check_input_length(message):
                return gr.update(interactive=len(message) >= self.MIN_CHARS)

            msg.submit(
                respond,
                inputs=[msg, chatbot],
                outputs=[msg, chatbot],
                api_name="submit"
            ).then(
                lambda: gr.update(interactive=False),
                outputs=submit_btn
            )

            submit_btn.click(
                respond,
                inputs=[msg, chatbot],
                outputs=[msg, chatbot],
                api_name="submit_btn"
            ).then(
                lambda: gr.update(interactive=False),
                outputs=submit_btn
            )

            msg.change(
                check_input_length,
                inputs=msg,
                outputs=submit_btn
            )

            clear.click(lambda: None, None, chatbot, queue=False)


        return interface

def main():
    chat_interface = ChatInterface()
    interface = chat_interface.create_interface()
    interface.launch(share=True)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d7e0dd8cefd9d86f38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d7e0dd8cefd9d86f38.gradio.live
